In [6]:
import numpy as np 
import pandas as pd 
import os 
import pickle 
import warnings
warnings.filterwarnings("ignore")

## Remove the empty files from the KSC MERLIN Cloud to Cloud Dataset

In [2]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
for yr in yrs:
    for mo in mos:
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            try:
                    temp = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            except:
                print('no data: '+file+' removing')
                os.remove(dir+file)


no data: KSCCC20170127.dat removing
no data: KSCCC20170126.dat removing
no data: KSCCC20170130.dat removing
no data: KSCCC20170124.dat removing
no data: KSCCC20170118.dat removing
no data: KSCCC20170119.dat removing
no data: KSCCC20170125.dat removing
no data: KSCCC20170131.dat removing
no data: KSCCC20170109.dat removing
no data: KSCCC20170121.dat removing
no data: KSCCC20170120.dat removing
no data: KSCCC20170112.dat removing
no data: KSCCC20170113.dat removing
no data: KSCCC20170111.dat removing
no data: KSCCC20170110.dat removing
no data: KSCCC20170128.dat removing
no data: KSCCC20170114.dat removing
no data: KSCCC20170101.dat removing
no data: KSCCC20170115.dat removing
no data: KSCCC20170129.dat removing
no data: KSCCC20170103.dat removing
no data: KSCCC20170117.dat removing
no data: KSCCC20170116.dat removing
no data: KSCCC20170102.dat removing
no data: KSCCC20170226.dat removing
no data: KSCCC20170225.dat removing
no data: KSCCC20170224.dat removing
no data: KSCCC20170220.dat r

In [8]:
data_dir = '/Users/brandonmcclung/Data/KSC_Weather_Archive/MERLIN_CC/'
yrs = ['2017']
mos = ['01','02','03','04','05','06','07','08','09','10','11','12']
i=0
for yr in yrs:
    for mo in mos:
        print(mo)
        first_file=True
        dir = data_dir+yr+'_CC/'+mo+'/'
        files = os.listdir(dir)
        for file in files:
            if first_file==True:
                first_file=False
                merlin_cc = pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False)
            else:
                print(file)
                if i<10:
                    merlin_cc = merlin_cc.append(pd.read_csv(dir+file,sep=' ',header=None,usecols=[0,2,6,8,12,15,20,22,24,26],low_memory=False))
        data_stor = '/Users/brandonmcclung/Data/pickles/'
        file = open(data_stor+yr+'_'+mo+'_merlin_cc_df.p','wb')
        pickle.dump(merlin_cc,file)
        file.close()    

01
KSCCC20170123.dat
02
KSCCC20170219.dat
KSCCC20170218.dat
KSCCC20170208.dat
KSCCC20170209.dat
KSCCC20170222.dat
KSCCC20170213.dat
KSCCC20170207.dat
KSCCC20170215.dat
KSCCC20170228.dat
KSCCC20170216.dat
03
KSCCC20170322.dat
KSCCC20170303.dat
04
KSCCC20170422.dat
KSCCC20170427.dat
KSCCC20170425.dat
KSCCC20170402.dat
KSCCC20170405.dat
05
KSCCC20170520.dat
KSCCC20170519.dat
KSCCC20170525.dat
KSCCC20170524.dat
KSCCC20170516.dat
KSCCC20170502.dat
KSCCC20170503.dat
KSCCC20170517.dat
KSCCC20170501.dat
KSCCC20170515.dat
KSCCC20170529.dat
KSCCC20170514.dat
KSCCC20170504.dat
KSCCC20170505.dat
KSCCC20170513.dat
06
KSCCC20170608.dat
KSCCC20170603.dat
KSCCC20170602.dat
KSCCC20170601.dat
KSCCC20170611.dat
KSCCC20170605.dat
KSCCC20170604.dat
KSCCC20170610.dat
KSCCC20170606.dat
KSCCC20170607.dat
07
KSCCC20170722.dat
KSCCC20170723.dat
KSCCC20170726.dat
KSCCC20170730.dat
KSCCC20170724.dat
KSCCC20170725.dat
KSCCC20170630.dat
KSCCC20170714.dat
KSCCC20170728.dat
KSCCC20170715.dat
KSCCC20170716.dat
KSCCC20

In [4]:
merlin_cc

,0,2,6,8,12,15,20,22,24,26
0,01/22/2017,18:50:20.4122000,29:51:44,-82:20:36,0.0,0,19.10,1.8,135,"62,63,64,66,67,68,70"
1,01/22/2017,18:50:49.8546000,29:24:09,-82:04:51,0.0,0,12.30,1.5,129,"63,64,65,66,67,70"
2,01/22/2017,18:52:53.8062000,29:55:26,-81:12:08,0.0,0,21.70,1.7,169,"64,65,66,68"
3,01/22/2017,18:53:29.6304000,29:54:06,-81:09:46,0.0,0,NaN,NaN,NaN,NaN
4,01/22/2017,18:57:00.7824000,29:44:55,-82:06:30,0.0,0,30.00,2.4,140,"62,64,65,66,68"
...,...,...,...,...,...,...,...,...,...,...
60126,02/15/2017,23:54:47.2298000,27:37:34,-80:09:24,0.0,0,13.70,01.60,150,"62,65,66,68"
60127,02/15/2017,23:56:09.0459000,27:39:16,-80:09:23,0.0,0,16.20,01.80,148,"62,65,66,68"
60128,02/15/2017,23:58:09.4442000,27:23:40,-79:55:08,0.0,0,14.20,01.90,143,"62,64,65,66"
60129,02/15/2017,23:58:09.4482000,27:24:05,-79:55:18,0.0,0,12.50,01.60,144,"62,64,65,66,68"
